In [8]:
import pandas as pd
import numpy as np
import math

sensor_id = 4004

# classification
diff_label = pd.read_csv('./test/'+str(sensor_id)+'classification.tsv', sep='\t')
label = np.array(diff_label)
x = label

# norm values
diff_norm = pd.read_csv('./test/'+str(sensor_id)+'normal.tsv', sep='\t')
diff_norm0 = np.array(diff_norm)
diff_norm0 = diff_norm0*(1-x)

# extreme values
diff_extream = pd.read_csv('./test/'+str(sensor_id)+'extreme.tsv', sep='\t')
diff_extream0 = np.array(diff_extream)
diff_extream0 = diff_extream0*x

# merge
comb = diff_norm0 + diff_extream0
test_set = pd.read_csv('./data/test_timestamps_24avg.tsv', sep='\t')
test_points = test_set["Start"]

if(str(sensor_id)[0]=='4'):
    trainX = pd.read_csv('./data'+'/reservoir_stor_'+str(sensor_id)+'_sof24.tsv', sep='\t')
elif(str(sensor_id)[0]=='6'):
    trainX = pd.read_csv('./data'+'/raingauge_byhour_'+str(sensor_id)+'_sof.tsv', sep='\t')
else:
    print("Error: the support sensor type is 4*** or 6****.")
    sys.exit()
    
i = 0
pre_yss_NEC = []
for ii in test_points:
    val_point = ii
    point = trainX[trainX["TSC_TSTAMP_UTC"]==ii].index.values[0]
    pre_y = trainX[point-1:point]["TSC_VALUE_F"].values.tolist()[0]
    pre_ys = []
    for j in range(72):
        diff = comb[i][j+1]
        pre_y = pre_y + diff
        pre_ys.append(pre_y)
    pre_yss_NEC.append(pre_ys)
    i += 1
    
# save the final data
aa = pd.DataFrame(data=pre_yss_NEC)
aa.to_csv("./test/"+str(sensor_id)+'final.tsv', sep='\t')    